In [4]:
import findspark

findspark.init()

from pyspark.sql import SparkSession

spark =  SparkSession.builder.getOrCreate()


In [6]:
data = [('srikanth','{"hair":"black","eye":"brown"}')]

schema = ['id','props']

df = spark.createDataFrame(data,schema)

df.show(truncate=False)

df.printSchema()

+--------+------------------------------+
|id      |props                         |
+--------+------------------------------+
|srikanth|{"hair":"black","eye":"brown"}|
+--------+------------------------------+

root
 |-- id: string (nullable = true)
 |-- props: string (nullable = true)



In [7]:
#from_json function allows you to convert maptype() or structtype()

from pyspark.sql.functions import from_json

help(from_json)

Help on function from_json in module pyspark.sql.functions:

from_json(col, schema, options=None)
    Parses a column containing a JSON string into a :class:`MapType` with :class:`StringType`
    as keys type, :class:`StructType` or :class:`ArrayType` with
    the specified schema. Returns `null`, in the case of an unparseable string.
    
    .. versionadded:: 2.1.0
    
    Parameters
    ----------
    col : :class:`~pyspark.sql.Column` or str
        string column in json format
    schema : :class:`DataType` or str
        a StructType or ArrayType of StructType to use when parsing the json column.
    
        .. versionchanged:: 2.3
            the DDL-formatted string is also supported for ``schema``.
    options : dict, optional
        options to control parsing. accepts the same options as the json datasource.
        See `Data Source Option <https://spark.apache.org/docs/latest/sql-data-sources-json.html#data-source-option>`_
        in the version you use.
    
        .. 

In [8]:
from pyspark.sql.types import MapType,StringType

MapTypeSchema = MapType(StringType(),StringType())

df1 = df.withColumn('propsMap',from_json(df.props,MapTypeSchema))

df1.show(truncate=False)

df1.printSchema()

+--------+------------------------------+-----------------------------+
|id      |props                         |propsMap                     |
+--------+------------------------------+-----------------------------+
|srikanth|{"hair":"black","eye":"brown"}|{hair -> black, eye -> brown}|
+--------+------------------------------+-----------------------------+

root
 |-- id: string (nullable = true)
 |-- props: string (nullable = true)
 |-- propsMap: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)



In [10]:
df2 = df1.withColumn('HAIR',df1.propsMap.hair).\
          withColumn('EYE',df1.propsMap.eye)

df2.show(truncate= False)


+--------+------------------------------+-----------------------------+-----+-----+
|id      |props                         |propsMap                     |HAIR |EYE  |
+--------+------------------------------+-----------------------------+-----+-----+
|srikanth|{"hair":"black","eye":"brown"}|{hair -> black, eye -> brown}|black|brown|
+--------+------------------------------+-----------------------------+-----+-----+



In [11]:
#from_json string into StructType

from pyspark.sql.functions import from_json
from pyspark.sql.types import StringType,StructField,StructType

structTypeSchema =  StructType([\
                               StructField('hair',StringType()),\
                               StructField('eye',StringType())])

df1 = df.withColumn('propsstruck',from_json(df.props,structTypeSchema))

df1.show()

df1.printSchema()

+--------+--------------------+--------------+
|      id|               props|   propsstruck|
+--------+--------------------+--------------+
|srikanth|{"hair":"black","...|{black, brown}|
+--------+--------------------+--------------+

root
 |-- id: string (nullable = true)
 |-- props: string (nullable = true)
 |-- propsstruck: struct (nullable = true)
 |    |-- hair: string (nullable = true)
 |    |-- eye: string (nullable = true)



In [13]:
df2 = df1.withColumn('hair',df1.propsstruck.hair) \
      .withColumn('eye',df1.propsstruck.eye)

df2.show()

df2.printSchema()

+--------+--------------------+--------------+-----+-----+
|      id|               props|   propsstruck| hair|  eye|
+--------+--------------------+--------------+-----+-----+
|srikanth|{"hair":"black","...|{black, brown}|black|brown|
+--------+--------------------+--------------+-----+-----+

root
 |-- id: string (nullable = true)
 |-- props: string (nullable = true)
 |-- propsstruck: struct (nullable = true)
 |    |-- hair: string (nullable = true)
 |    |-- eye: string (nullable = true)
 |-- hair: string (nullable = true)
 |-- eye: string (nullable = true)

